# Setup

In [ ]:
import pandas as pd
import geopandas as gpd
import geodatasets
import matplotlib.pyplot as plt
import numpy as np

# Load the Excel file and specific sheet
excel_file = 'CAT_26042024_SectorAssessmentData_DataExplorer.xlsx'
sheet_name = 'Data'
df = pd.read_excel(excel_file, sheet_name=sheet_name, engine='openpyxl')

# Add my own column names
df.columns = ['Type', 'Description', 'ISO_A3', 'Year', 'Timeline', 'Value', 'Unit']
# Save the DataFrame to a CSV file
df.to_csv('output3.csv', index=False)

## Data Exploration

In [ ]:
df.head()

In [ ]:
df['ISO_A3'].unique()

In [ ]:
df.info()

### Identify What Types there are

In [ ]:
df['Type'].unique()

### Identify timelines

In [ ]:
df['Timeline'].unique()

## Get Valid iso_a3 values
### Remove regional or group level entities, like EU and World

In [ ]:
import pycountry

country_column = df['ISO_A3']

# Get a list of valid ISO Alpha-3 country codes
valid_iso_a3 = {country.alpha_3 for country in pycountry.countries}
print(valid_iso_a3)

In [ ]:
# Filter rows with valid ISO codes
valid_countries = country_column[country_column.isin(valid_iso_a3)]

# Preview valid countries
valid_countries.unique()

In [ ]:
# Compare valid countries to what I removed
country_column.unique()

In [ ]:
filtered_df_country = df[df['ISO_A3'].isin(valid_iso_a3)]

In [ ]:
filtered_df_country['ISO_A3'].unique()

### Get World Data

In [ ]:
world = gpd.read_file("ne_110m_admin_0_countries.shp")

### Plot World Map

In [ ]:
world.plot(figsize=(15, 10), edgecolor='black')
plt.title("Basic World Map", fontsize=20)
plt.show()

## Filter Data

In [ ]:
filtered_df_country = df[(df['Type'] == 'Electricity') & (df['Timeline'] == 'historic')]
filtered_df_country.to_csv('elect.csv', index=False)
filtered_data_country_desc = filtered_df_country[filtered_df_country['Description'].str.contains('Emissions intensity of electricity generation', case=False, na=False)]

### Electricity is the only remaining Type

In [ ]:
filtered_data_country_desc['Type'].unique()

In [ ]:
# Check columns in the world DataFrame
print(world.columns)

# Check columns in the filtered_df_country DataFrame
print(filtered_data_country_desc.columns)
print(filtered_data_country_desc.head)

## France == -99

### Our Dataset is correct

In [ ]:
csv_data = pd.read_csv('output3.csv')
csv_data["ISO_A3"].unique()

In [ ]:
# FRA values are not showing up in map
print(df[df["ISO_A3"].str.contains('FRA', na=False, case=False)])

In [ ]:
filtered_data_country_desc['ISO_A3'].unique()

### Natural World Dataset is not correct

In [ ]:
world['ISO_A3'].unique()

In [ ]:
world["ISO_A3"] = world["ISO_A3"].replace("-99", "FRA")

## Merge world dataset and emissions dataset

In [ ]:
mergedIntensity = world.merge(filtered_data_country_desc, how='left', on='ISO_A3')
mergedIntensity['ISO_A3'] = np.where(mergedIntensity['Value'].isna(), np.nan, mergedIntensity['ISO_A3'])
mergedIntensity['ISO_A3'].unique()

## Plot Intensity Map

In [ ]:
mergedIntensity.plot(
    column='Value',
    cmap='Reds',   # Adjust colormap as needed
    legend=True,
    figsize=(15, 10),
    missing_kwds={
        "color": "darkgray",  # Color for missing data
        "label": "No Data"    # Label for missing data in legend
    }
)

plt.title("Electricity Intensity Emissions Map (No Data Gray, g CO2 / KWH)")
plt.show()

# Share of Coal Percentage in Electricity Generation in World

In [ ]:
filtered_data1 = df[
    (df["ISO_A3"] == "World") &
    (df["Description"] == "Share of coal in electricity generation")
]
filtered_data1 = filtered_data1[(filtered_data1['Timeline'] == 'historic')]

In [ ]:
filtered_data1 = filtered_data1.sort_values(by="Year")

In [ ]:
# Plot the data
plt.figure(figsize=(10, 6))
plt.plot(filtered_data1["Year"], filtered_data1["Value"], marker="o", label="Share of Coal")

# Add labels and title
plt.title("Share of Coal in Electricity Generation (World)", fontsize=16)
plt.xlabel("Year", fontsize=14)
plt.ylabel("Share (%)", fontsize=14)
plt.grid(True)
plt.legend()

# Show the plot
plt.show()